# BluWave Challenge

The goal of this challenge is to determine one or more outputs from 4 wind turbines are stationary.

In [1]:
# import modules

import pandas as pd
import numpy as np
import os
import sys
import random
import copy
import cPickle as pickle
import datetime

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import colorlover as cl
import matplotlib.pyplot as plt 

init_notebook_mode(connected=True)

%reload_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.4f}'.format

In [30]:
# Load data

data = pd.read_excel("Data.xlsx", sheet_name=None).values()[0]

In [58]:
pw_cols = ['PW1', 'PW2', 'PW3', 'PW4']

## Explore/Prepare Data Set

First we take a look at the size, column types and the first couple rows of the dataframe, shown below.  The wind turbine outputs are the columns PW1, PW2, PW3 and PW4. There is a TimeStamp column which contains the epoch timestamps (number of seconds since Jan 1, 1970) for each sample. 

In [20]:
data.shape

(106560, 15)

In [103]:
data.dtypes

TimeStamp              float64
Date                    object
Time                    object
Wind Speed             float64
Wind Dir               float64
Temp                   float64
Unnamed: 6             float64
Unnamed: 7              object
Unnamed: 8              object
PW1                     object
PW2                     object
PW3                     object
PW4                     object
Unnamed: 13             object
 Wind Farm MW Value    float64
dtype: object

In [8]:
data.head()

TimeStamp          Date       Time  Wind Speed  Wind Dir    Temp  \
0 1,501,545,600.0000   "Aug 01/17"   00:00:00      8.6741  211.8667 20.0000   
1 1,501,545,900.0000   "Aug 01/17"   00:05:00      8.2880  213.7232 20.0000   
2 1,501,546,200.0000   "Aug 01/17"   00:10:00      8.5543  216.4733 20.0000   
3 1,501,546,500.0000   "Aug 01/17"   00:15:00      8.5694  218.5832 20.0000   
4 1,501,546,800.0000   "Aug 01/17"   00:20:00      8.7656  218.0367 20.0000   

   Unnamed: 6 Unnamed: 7 Unnamed: 8        PW1        PW2        PW3  \
0         nan        NaN        NaN 1,106.2140 1,071.4309 1,165.2189   
1         nan        NaN        NaN 1,023.2526   920.8148 1,126.9757   
2         nan        NaN        NaN 1,066.6329 1,173.6629 1,116.1848   
3         nan        NaN        NaN 1,067.4646 1,043.5497 1,171.3774   
4         nan        NaN        NaN 1,092.3378 1,058.1656 1,197.3882   

         PW4 Unnamed: 13   Wind Farm MW Value  
0 1,079.5228         NaN           4,407.5907  
1   955.0125         NaN           3,979.1336  
2   950.9738         NaN           4,312.0781  
3 1,153.1642         NaN           4,359.9587  
4 1,243.6106         NaN           4,602.9744

In terms of time ranges, the samples start and end at the following dates:

In [4]:
'start: ' + data['Date'].iloc[0] + ' end: ' + data['Date'].iloc[-1]

u'start:  "Aug 01/17" end:  "Aug 05/18"'

So there's roughly a year's worth of data. In addition the time interval between each pair of successive points appears to be 5 minutes.

Its strange, however that the PW columns are object types since they appear to be floats. The unique values for the columns are shown below as well as the number of null values

In [110]:
for w in pw_cols:
    print np.unique(data[w])

[-47.165427 -40.573715 -40.265819 ..., 3008.55566 3011.689958 u' ']
[-51.420665 -48.12117 -46.364245 ..., 3007.130397 3009.956445 u' ']
[-46.234508 -45.469031 -44.202472 ..., 3008.425407 3008.767463 u' ']
[-55.90785 -49.78317 -49.744794 ..., 3007.361389 3007.539973 u' ']


In [46]:
data.isnull().sum()

TimeStamp                 11
Date                       0
Time                       0
Wind Speed                11
Wind Dir                  11
Temp                      11
Unnamed: 6             96205
Unnamed: 7             96205
Unnamed: 8             96205
PW1                       11
PW2                       11
PW3                       11
PW4                       11
Unnamed: 13            96205
 Wind Farm MW Value       11
dtype: int64

There are strings mixed in with the floats and each turbine has 11 points of missing data. Combining NaN and string values, we get the following missing values for each turbine:

In [111]:
data.isnull().sum() + (data == ' ').sum()

TimeStamp                  11
Date                        0
Time                        0
Wind Speed                 11
Wind Dir                   11
Temp                       11
Unnamed: 6              96205
Unnamed: 7             106560
Unnamed: 8             106560
PW1                      1778
PW2                      1830
PW3                      1143
PW4                      1133
Unnamed: 13            106560
 Wind Farm MW Value        11
dtype: int64

These 1000 or so missing values for each turbine represents about 0.9% of the data.  This isn't a huge amount but could become a reocurring issue.

In [120]:
n = ((data == ' ') | (data.isnull()))
len(list(data.loc[n['PW1'], ['Date', 'Time']].groupby('Date')))

32

In [74]:
miss_data = data.loc[data.loc[:, pw_cols].isnull().any(1),:]
miss_data_inds = miss_data.index
miss_data

TimeStamp          Date       Time  Wind Speed  Wind Dir  Temp  \
96194        nan   "Jul 01/18"   00:05:00         nan       nan   nan   
96195        nan   "Jul 01/18"   00:10:00         nan       nan   nan   
96196        nan   "Jul 01/18"   00:15:00         nan       nan   nan   
96197        nan   "Jul 01/18"   00:20:00         nan       nan   nan   
96198        nan   "Jul 01/18"   00:25:00         nan       nan   nan   
96199        nan   "Jul 01/18"   00:30:00         nan       nan   nan   
96200        nan   "Jul 01/18"   00:35:00         nan       nan   nan   
96201        nan   "Jul 01/18"   00:40:00         nan       nan   nan   
96202        nan   "Jul 01/18"   00:45:00         nan       nan   nan   
96203        nan   "Jul 01/18"   00:50:00         nan       nan   nan   
96204        nan   "Jul 01/18"   00:55:00         nan       nan   nan   

       Unnamed: 6 Unnamed: 7 Unnamed: 8  PW1  PW2  PW3  PW4 Unnamed: 13  \
96194         nan        NaN        NaN  NaN  NaN  NaN  NaN         NaN   
96195         nan        NaN        NaN  NaN  NaN  NaN  NaN         NaN   
96196         nan        NaN        NaN  NaN  NaN  NaN  NaN         NaN   
96197         nan        NaN        NaN  NaN  NaN  NaN  NaN         NaN   
96198         nan        NaN        NaN  NaN  NaN  NaN  NaN         NaN   
96199         nan        NaN        NaN  NaN  NaN  NaN  NaN         NaN   
96200         nan        NaN        NaN  NaN  NaN  NaN  NaN         NaN   
96201         nan        NaN        NaN  NaN  NaN  NaN  NaN         NaN   
96202         nan        NaN        NaN  NaN  NaN  NaN  NaN         NaN   
96203         nan        NaN        NaN  NaN  NaN  NaN  NaN         NaN   
96204         nan        NaN        NaN  NaN  NaN  NaN  NaN         NaN   

        Wind Farm MW Value  
96194                  nan  
96195                  nan  
96196                  nan  
96197                  nan  
96198                  nan  
96199                  nan  
96200                  nan  
96201                  nan  
96202                  nan  
96203                  nan  
96204                  nan

We can see these missing values are consecutive and occured on July 1rst for an hour for all wind turbines.  It appears to be some kind of blackout in the data collector  - perhaps the collectors were powered down for a yearly maintenance. We could assume that these values are missing at random - i.e. their missing-ness is not related to parameters of interest.

We'll need to either remove these values completely or impute.  Given that these values represent only 0.01% of the data and occurred over a single hour, its unlikely that the method we use will impact any tests for stationarity significantly. However, this could become a recurring issue we need to deal with.

There are a couple basic imputing methods specific to time series we could consider:

1. Last observation carried forward (LOCF)
2. Next observation carried backward (NOCB)
3. Linear interpolation
4. Spline interpolation
5. Moving average

We'll first plot the data from the turbines before and after the missing chunk.  We used the 12 hours before and after since trends in weather will appear in this time frame.

In [61]:
st_missing = 96194
end_missing = 96204

# Get indices of points 12h before gap and 12h after
inds = range(st_missing - (12*12), st_missing + 1) + range(end_missing, end_missing + (12*12))

iplot(go.Figure(data = [go.Scatter(x = data['TimeStamp'].iloc[inds], y = data[w].iloc[inds], 
                                   mode = 'lines', name = w) for w in pw_cols]))

The gap appears to occur during a period of high/variable winds so using LCOF and NOCB would create a 'jump' in the data. Linear interpolation wouldn't capture the variability going on in the data during that period. Spline interpolation would capture too much noise. Moving average could help to impute the general trend.

In [72]:
rolldata = data.loc[:, pw_cols].iloc[inds, :].rolling(window = 10).mean()
iplot(go.Figure(data = [go.Scatter(x = range(rolldata.shape[0]), y = rolldata[w], 
                                   mode = 'lines', name = w) for w in pw_cols]))

In [62]:
44.0 * 100.0 / (data.shape[0] * 4)

0.010322822822822823

Now we'll plot the output against time for each wind turbine using plotly. Given that there are so many points, we'll pick the first point of every day, just to get a sense of the data. 

In [21]:
pt_interval = int(3600.0 * 24.0 / (5.0 * 60))
for w in ['PW1', 'PW2', 'PW3', 'PW4']:
    iplot(go.Figure(data = [go.Scatter(x = data['TimeStamp'].iloc[::pt_interval], 
                                   y = data[w].iloc[::288], mode = 'lines')],
               layout = go.Layout(yaxis = dict(range = [0, 4000]), title = w)))

At first glance, there doesn't appear to be a trend in these time series.

We can also plot summary statistics over time for each time series.  Here, we plot the mean and variance for each day.

In [100]:
clean_data = data.drop(miss_data_inds).reset_index()
pd.to_numeric(clean_data.loc[:, pw_cols]).groupby(clean_data['Date']).mean()

TypeError: arg must be a list, tuple, 1-d array, or Series

In [108]:
clean_data.loc[clean_data['PW1'] == ' ', :]

index          TimeStamp          Date       Time  Wind Speed  \
1612    1612 1,502,029,200.0000   "Aug 06/17"   14:20:00      9.3233   
1613    1613 1,502,029,500.0000   "Aug 06/17"   14:25:00      8.6513   
1614    1614 1,502,029,800.0000   "Aug 06/17"   14:30:00      8.7041   
1615    1615 1,502,030,100.0000   "Aug 06/17"   14:35:00      9.0740   
1616    1616 1,502,030,400.0000   "Aug 06/17"   14:40:00      8.8560   
1617    1617 1,502,030,700.0000   "Aug 06/17"   14:45:00      9.7810   
1618    1618 1,502,031,000.0000   "Aug 06/17"   14:50:00      9.6160   
1619    1619 1,502,031,300.0000   "Aug 06/17"   14:55:00      9.1280   
1620    1620 1,502,031,600.0000   "Aug 06/17"   15:00:00      9.3431   
1621    1621 1,502,031,900.0000   "Aug 06/17"   15:05:00      9.0097   
5554    5554 1,503,211,800.0000   "Aug 20/17"   06:50:00      5.0000   
5555    5555 1,503,212,100.0000   "Aug 20/17"   06:55:00      5.0000   
5556    5556 1,503,212,400.0000   "Aug 20/17"   07:00:00      5.0000   
5557    5557 1,503,212,700.0000   "Aug 20/17"   07:05:00      5.0000   
5558    5558 1,503,213,000.0000   "Aug 20/17"   07:10:00      5.0000   
5559    5559 1,503,213,300.0000   "Aug 20/17"   07:15:00      5.0000   
5560    5560 1,503,213,600.0000   "Aug 20/17"   07:20:00      5.0000   
5561    5561 1,503,213,900.0000   "Aug 20/17"   07:25:00      5.0000   
5562    5562 1,503,214,200.0000   "Aug 20/17"   07:30:00      5.0000   
5563    5563 1,503,214,500.0000   "Aug 20/17"   07:35:00      5.0000   
5564    5564 1,503,214,800.0000   "Aug 20/17"   07:40:00      5.0000   
5565    5565 1,503,215,100.0000   "Aug 20/17"   07:45:00      5.0000   
5566    5566 1,503,215,400.0000   "Aug 20/17"   07:50:00      5.0000   
5567    5567 1,503,215,700.0000   "Aug 20/17"   07:55:00      5.0000   
5568    5568 1,503,216,000.0000   "Aug 20/17"   08:00:00      5.0000   
5569    5569 1,503,216,300.0000   "Aug 20/17"   08:05:00      5.0000   
5570    5570 1,503,216,600.0000   "Aug 20/17"   08:10:00      5.0000   
5571    5571 1,503,216,900.0000   "Aug 20/17"   08:15:00      5.0000   
5572    5572 1,503,217,200.0000   "Aug 20/17"   08:20:00      5.0000   
5573    5573 1,503,217,500.0000   "Aug 20/17"   08:25:00      5.0000   
...      ...                ...           ...        ...         ...   
86163  86163 1,527,394,500.0000   "May 27/18"   04:15:00      6.3000   
86164  86164 1,527,394,800.0000   "May 27/18"   04:20:00      6.3000   
86165  86165 1,527,395,100.0000   "May 27/18"   04:25:00      6.3000   
86166  86166 1,527,395,400.0000   "May 27/18"   04:30:00      6.3000   
86167  86167 1,527,395,700.0000   "May 27/18"   04:35:00      6.3000   
86168  86168 1,527,396,000.0000   "May 27/18"   04:40:00      6.3000   
86169  86169 1,527,396,300.0000   "May 27/18"   04:45:00      6.3000   
86170  86170 1,527,396,600.0000   "May 27/18"   04:50:00      6.3000   
86171  86171 1,527,396,900.0000   "May 27/18"   04:55:00      6.3000   
86172  86172 1,527,397,200.0000   "May 27/18"   05:00:00      6.3000   
86173  86173 1,527,397,500.0000   "May 27/18"   05:05:00      6.3000   
86174  86174 1,527,397,800.0000   "May 27/18"   05:10:00      6.3000   
86175  86175 1,527,398,100.0000   "May 27/18"   05:15:00      6.3000   
86176  86176 1,527,398,400.0000   "May 27/18"   05:20:00      6.3000   
86177  86177 1,527,398,700.0000   "May 27/18"   05:25:00      6.3000   
86178  86178 1,527,399,000.0000   "May 27/18"   05:30:00      6.3000   
86179  86179 1,527,399,300.0000   "May 27/18"   05:35:00      6.3000   
86180  86180 1,527,399,600.0000   "May 27/18"   05:40:00      6.3000   
86181  86181 1,527,399,900.0000   "May 27/18"   05:45:00      6.3000   
86182  86182 1,527,400,200.0000   "May 27/18"   05:50:00      6.3000   
86183  86183 1,527,400,500.0000   "May 27/18"   05:55:00      6.3000   
86184  86184 1,527,400,800.0000   "May 27/18"   06:00:00      6.3000   
91142  91142 1,528,888,200.0000   "Jun 13/18"   11:10:00      6.0000   
91143  91143 1,528,888,500.0000   "Jun 13/18"  

In [45]:
data.isnull().sum()

TimeStamp                 11
Date                       0
Time                       0
Wind Speed                11
Wind Dir                  11
Temp                      11
Unnamed: 6             96205
Unnamed: 7             96205
Unnamed: 8             96205
PW1                       11
PW2                       11
PW3                       11
PW4                       11
Unnamed: 13            96205
 Wind Farm MW Value       11
dtype: int64

In [44]:
day_means = data.groupby('Date').mean()
day_means

TimeStamp  Wind Speed  Wind Dir    Temp  Unnamed: 6  \
Date                                                                        
 "Apr 01/18" 1,522,583,850.0000      9.4738  208.1105  4.2292         nan   
 "Apr 02/18" 1,522,670,250.0000      5.9605  222.7329  1.3277         nan   
 "Apr 03/18" 1,522,756,650.0000      4.2754  180.0378  2.7087         nan   
 "Apr 04/18" 1,522,843,050.0000      6.7455  130.4163  3.5698         nan   
 "Apr 05/18" 1,522,929,450.0000     11.9473  250.7715  1.0854         nan   
 "Apr 06/18" 1,523,015,850.0000     10.5332  243.7872 -1.1614         nan   
 "Apr 07/18" 1,523,102,250.0000      5.6646  161.5783  1.9786         nan   
 "Apr 08/18" 1,523,188,650.0000      5.0219  141.0974  0.4599         nan   
 "Apr 09/18" 1,523,275,050.0000     12.0483  326.2896 -0.0067         nan   
 "Apr 10/18" 1,523,361,450.0000      6.4960  241.1790  1.6558         nan   
 "Apr 11/18" 1,523,447,850.0000      4.9007  157.9636  3.2083         nan   
 "Apr 12/18" 1,523,534,250.0000      5.6084  188.4524  4.3379         nan   
 "Apr 13/18" 1,523,620,650.0000      9.1368  213.4470  4.9153         nan   
 "Apr 14/18" 1,523,707,050.0000     10.4396  293.1190  2.3523         nan   
 "Apr 15/18" 1,523,793,450.0000      6.5382  207.3591 -1.0871         nan   
 "Apr 16/18" 1,523,879,850.0000      4.2873   67.8243  0.9523         nan   
 "Apr 17/18" 1,523,966,250.0000     11.0632  120.7561  4.4425         nan   
 "Apr 18/18" 1,524,052,650.0000      8.2823  214.6610  4.8754         nan   
 "Apr 19/18" 1,524,139,050.0000      5.8744  186.5296  5.7992         nan   
 "Apr 20/18" 1,524,225,450.0000      7.7009  171.5036  1.5055         nan   
 "Apr 21/18" 1,524,311,850.0000      5.0010  235.8371  3.5288         nan   
 "Apr 22/18" 1,524,398,250.0000      5.5006  170.2172  5.3085         nan   
 "Apr 23/18" 1,524,484,650.0000      5.2492  250.7161  8.8662         nan   
 "Apr 24/18" 1,524,571,050.0000     10.1069  213.9433  9.8560         nan   
 "Apr 25/18" 1,524,657,450.0000      9.7707  203.1986 10.8544         nan   
 "Apr 26/18" 1,524,743,850.0000     11.2526  153.7095 12.6970         nan   
 "Apr 27/18" 1,524,830,250.0000      5.8043   79.9285  7.3189         nan   
 "Apr 28/18" 1,524,916,650.0000      5.8733   80.9559  5.8378         nan   
 "Apr 29/18" 1,525,003,050.0000      5.8043   79.9285  7.3189         nan   
 "Apr 30/18" 1,525,089,450.0000      5.8733   80.9559  5.8378         nan   
...                         ...         ...       ...     ...         ...   
 "Sep 01/17" 1,504,267,050.0000      7.4227  203.0000 12.8763         nan   
 "Sep 02/17" 1,504,353,450.0000      6.4310  203.0000 12.3842         nan   
 "Sep 03/17" 1,504,439,850.0000      4.9817  203.0000 16.0339         nan   
 "Sep 04/17" 1,504,526,250.0000     10.2186  203.0000 16.4878         nan   
 "Sep 05/17" 1,504,612,650.0000     11.0654  203.0000 19.0653         nan   
 "Sep 06/17" 1,504,699,050.0000      7.6481  203.0000 21.3674         nan   
 "Sep 07/17" 1,504,785,450.0000     10.3769  203.0000 20.5615         nan   
 "Sep 08/17" 1,504,871,850.0000      7.6044  203.0000 17.6461         nan   
 "Sep 09/17" 1,504,958,250.0000      5.1755  203.0000 16.7561         nan   
 "Sep 10/17" 1,505,044,650.0000      6.7624  203.0000 14.9632         nan   
 "Sep 11/17" 1,505,131,050.0000     10.4336  203.0000 14.5429         nan   
 "Sep 12/17" 1,505,217,450.0000      6.3877  203.0000 15.9191         nan   
 "Sep 13/17" 1,505,303,850.0000      5.1945  203.0000 19.0212         nan   
 "Sep 14/17" 1,505,390,250.0000      6.8062  203.0000 20.7283         nan   
 "Sep 15/17" 1,505,476,650.0000      5.0284  203.0000 16.6678         nan   
 "Sep 16/17" 1,505,563,050.0000      7.1122  203.0000 16.8325         nan   
 "Sep 17/17" 1,505,649,450.0000      3.3435  203.0000 15.3135         nan   
 "Sep 18/17" 1,505,735,850.0000      4.1534  203.0000 16.6218         nan   
 "Sep 19/17" 1,505,822,250.0000      4.7882  203.0000 18.2691         nan   
 "Sep 20/17" 1,505

In [22]:
date_time = data.apply(lambda x: x['Date'].split('"') + x['Time'], axis = 0)
#datetime.datetime.strptime(data['Date'].iloc[0].split('"')[1], "%b %d/%y")

KeyError: ('Date', u'occurred at index TimeStamp')

In [16]:
data['Date'].iloc[0].split('"')[1]

[u' ', u'Aug 01/17', u'']

In [18]:
data['Time'].iloc[0]

u' 00:00:00'